# 1. Reglas de asociacion

Como paso previo, y  puesto que el notebook de jupyter se ha instalado y ejecutado en windows, he instalado una implementación de apriori siguiente: https://pypi.python.org/pypi/apriori/1.0.0

por ese motivo, es posible que los métodos no sean identicos a los que se han utilizado en la práctica de clases, pero el resultado es similar. He tenido que investigar ya que la librería no está documentada. 

In [11]:
import csv
import apriori

groceries=[]
groceries_file=csv.reader(open("groceries.csv","rb"))
for row in groceries_file:
    groceries.append(row)



Utilizamos el algoritmo para detectar el soporte de la aparicion de los productos dentro de cada registro de compra y que superen el 0,05, es decir, de aquellos productos y combinaciones que al menos aparezcan en el 5% de las compras.

In [16]:
F, soporte = apriori.apriori(groceries, minSupport=0.05)
soporte

{frozenset({'pot plants'}): 0.01728520589730554,
 frozenset({'sausage', 'tropical fruit'}): 0.013929842399593289,
 frozenset({'other vegetables', 'pork'}): 0.021657346212506354,
 frozenset({'frankfurter'}): 0.058973055414336555,
 frozenset({'frankfurter', 'other vegetables'}): 0.016471784443314694,
 frozenset({'beef'}): 0.05246568378240976,
 frozenset({'pip fruit', 'whipped/sour cream'}): 0.009252669039145907,
 frozenset({'napkins', 'other vegetables'}): 0.01443823080833757,
 frozenset({'specialty cheese'}): 0.008540925266903915,
 frozenset({'napkins', 'whipped/sour cream'}): 0.007219115404168785,
 frozenset({'canned beer', 'pork'}): 0.0029486527707168276,
 frozenset({'fruit/vegetable juice', 'pip fruit'}): 0.009557702084392475,
 frozenset({'bottled beer', 'pip fruit'}): 0.005897305541433655,
 frozenset({'bottled beer', 'pork'}): 0.005185561769191663,
 frozenset({'liquor'}): 0.011082867310625319,
 frozenset({'coffee', 'domestic eggs'}): 0.00498220640569395,
 frozenset({'other vegetable

Una vez obtenido el soporte, buscamos aquellas combinaciones que dado A se cumple la compra de B en mas del 25% de las veces

In [21]:
H = apriori.generateRules(F, soporte, minConf=0.25)
H

frozenset(['yogurt']) --> frozenset(['whole milk']) conf: 0.401603498542
frozenset(['rolls/buns']) --> frozenset(['whole milk']) conf: 0.307904919845
frozenset(['other vegetables']) --> frozenset(['whole milk']) conf: 0.38675775092
frozenset(['whole milk']) --> frozenset(['other vegetables']) conf: 0.292877039395


[(frozenset({'yogurt'}), frozenset({'whole milk'}), 0.40160349854227406),
 (frozenset({'rolls/buns'}), frozenset({'whole milk'}), 0.30790491984521834),
 (frozenset({'other vegetables'}),
  frozenset({'whole milk'}),
  0.38675775091960063),
 (frozenset({'whole milk'}),
  frozenset({'other vegetables'}),
  0.2928770393951452)]